In [365]:
from __future__ import division
import numpy as np
import scipy as sc
from itertools import product
import time
import matplotlib.pyplot as plt
import PIL
from numpy import log10
import random
from math import factorial
from scipy.stats import linregress, gaussian_kde, skew
from scipy import stats
from scipy.spatial import distance
import warnings
import pandas as pd
import re
import os
import math
from collections import Counter
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.stats.outliers_influence import summary_table
import statsmodels.api as sm
from scipy.optimize import linear_sum_assignment

warnings.filterwarnings('ignore')

%config InlineBackend.figure_formats = ['svg']
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}

pd.set_option('display.max_columns', None)


In [366]:
def generate_cum_dists(n_obs, n_bins, cum=True):
    """Generate all possible discrete distributions for a given number of 
        observations distributed across a given number of bins
    """
    
    def partitions(n, k):
        if k == 1:
            yield (n,)
        else:
            for i in range(n + 1):
                for result in partitions(n - i, k - 1):
                    yield (i,) + result

    for combo in partitions(n_obs, n_bins):
        if cum:
            cum_dist = [sum(combo[:i + 1]) for i in range(n_bins)]
            yield cum_dist
        else:
            yield combo

            
def get_P(f):
    k = len(f)
    n = sum(f)
    
    # 1. Sum of the normalized cumulative values (frequencies, counts, etc.) -- P, the positional average of mass from right to left
    # Basically adding up the discrete cumulative classes and dividing by the max.
    pn = [sum(f[:ii+1])/n for ii in range(k)]
    P = np.sum(pn)
    
    # 2. Distance of P from the max discrete class -- D = P - 1
    D = P - 1 
    
    # 3. D as a fraction of the maximum distance scaled between 0 and 1
    S = D/(k - 1)
    
    # 4. P as the positional average of mass from left to right ... based cumulative distribution
    P_LtR_1 = k - P + 1
    
    # 5. P as the positional average of mass from left to right ... based on frequency distribution
    #positions = np.arange(1, k + 1)  # 1-based positions
    #P_LtR_2 = np.sum(positions * f) / sum(f)
    
    P_LtR_2 = 0
    for i, fi in enumerate(f):
        P_LtR_2 += (i + 1)*fi/sum(f)
    
    return P, D, S, P_LtR_1, P_LtR_2


def get_DS(f):
    k = len(f)
    n = sum(f)
    
    
    #F = [sum(f[:ii+1])/n for ii in range(len(f))]
    #DS = (np.sum(np.array(F)/(n)) - 1) / (k - 1)
    
    DS = 0
    for i, fi in enumerate(f):
        DS += (i + 1)*fi/sum(f)
        
    DS = DS + k - 1
    
    return DS


def RDS(p, q):
    
    rds_p = get_DS(p)
    rds_q = get_DS(q)
    
    return rds_q - rds_p


In [ ]:
def get_DS(f):
    k = len(f)
    n = sum(f)
    
    
    #F = [sum(f[:ii+1])/n for ii in range(len(f))]
    #DS = (np.sum(np.array(F)/(n)) - 1) / (k - 1)
    
    DS = 0
    for i, fi in enumerate(f):
        DS += (i + 1)*fi/sum(f)
        
    DS = DS + k - 1
    
    return DS


def RDS(p, q):
    
    rds_p = get_DS(p)
    rds_q = get_DS(q)
    
    return rds_q - rds_p

In [363]:
def generate_cum_dists(n_obs, n_bins, cum=True):
    """Generate all possible discrete distributions for a given number of 
        observations distributed across a given number of bins
    """
    
    def partitions(n, k):
        if k == 1:
            yield (n,)
        else:
            for i in range(n + 1):
                for result in partitions(n - i, k - 1):
                    yield (i,) + result

    for combo in partitions(n_obs, n_bins):
        if cum:
            cum_dist = [sum(combo[:i + 1]) for i in range(n_bins)]
            yield cum_dist
        else:
            yield combo


# Generate distributions one at a time and add to a list
_set = []
for dist in generate_cum_dists(n, k, cum=False):
    _set.append(dist)

n = 5
k = 3

P1_ls = []
P1_scaled_ls = []
P2_ls = []
P2_scaled_ls = []

for i, f in enumerate(_set):

    # Use the frequency distribution
    P1_scaled = 0
    P1 = 0
        
    for indx, fi in enumerate(f):
        i = indx + 1
        
        P1 += (i*fi) / sum(f)
        
        z = 1 + 1/
        P1_scaled += i*(fi/sum(f))**z
    
    P1_ls.append(P1)
    P1_scaled_ls.append(P1_scaled)
    
    # Use the cumulative frequency distribution
    z = 1 + 1/n
    P2 = [sum(f[:ii+1]) for ii in range(len(f))]
    P2 = (np.array(P2)/n)
    P2_ls.append(k + 1 - sum(P2)) # P1 should equal P2
    
    P2 = [sum(f[:ii+1]) for ii in range(len(f))]
    P2 = (np.array(P2)/n)**z
    P2_scaled_ls.append(k + 1 - sum(P2))
    

#P1_scaled_ls, P1_ls, P2_scaled_ls, P2_ls, _set = zip(*sorted(zip(P1_scaled_ls, P1_ls, P2_scaled_ls, P2_ls, _set)))
P1_ls, P1_scaled_ls, P2_scaled_ls, P2_ls, _set = zip(*sorted(zip(P1_ls, P1_scaled_ls, P2_scaled_ls, P2_ls, _set)))

for i, P1 in enumerate(P1_ls):
    print('f:', _set[i], '| P1 =', round(P1_ls[i],4), '| P2 =', round(P2_ls[i],4), '| scaled P1 =', round(P1_scaled_ls[i],6),  '| scaled P2 =', round(P2_scaled_ls[i],6))

print('\n')
print(len(P1_ls), 'members of the feasible set for n and k')



f: (5, 0, 0) | P1 = 1.0 | P2 = 1.0 | scaled P1 = 1.0 | scaled P2 = 1.0
f: (4, 1, 0) | P1 = 1.2 | P2 = 1.2 | scaled P1 = 1.054994 | scaled P2 = 1.234918
f: (4, 0, 1) | P1 = 1.4 | P2 = 1.4 | scaled P1 = 1.19995 | scaled P2 = 1.469836
f: (3, 2, 0) | P1 = 1.4 | P2 = 1.4 | scaled P1 = 1.207771 | scaled P2 = 1.458272
f: (3, 1, 1) | P1 = 1.6 | P2 = 1.6 | scaled P1 = 1.266508 | scaled P2 = 1.69319
f: (2, 3, 0) | P1 = 1.6 | P2 = 1.6 | scaled P1 = 1.416478 | scaled P2 = 1.666979
f: (3, 0, 2) | P1 = 1.8 | P2 = 1.8 | scaled P1 = 1.540792 | scaled P2 = 1.916543
f: (1, 4, 0) | P1 = 1.8 | P2 = 1.8 | scaled P1 = 1.67512 | scaled P2 = 1.855044
f: (2, 2, 1) | P1 = 1.8 | P2 = 1.8 | scaled P1 = 1.433932 | scaled P2 = 1.901897
f: (2, 1, 2) | P1 = 2.0 | P2 = 2.0 | scaled P1 = 1.621997 | scaled P2 = 2.12525
f: (1, 3, 1) | P1 = 2.0 | P2 = 2.0 | scaled P1 = 1.66328 | scaled P2 = 2.089962
f: (0, 5, 0) | P1 = 2.0 | P2 = 2.0 | scaled P1 = 2.0 | scaled P2 = 2.0
f: (1, 2, 2) | P1 = 2.2 | P2 = 2.2 | scaled P1 = 1.81

# Figure


In [364]:
n_obs = 10
n_bins = 5
fig = plt.figure(figsize=(7, 4))
   
################  TOP ROW  ###########

_set = []
for dist in generate_cum_dists(n_obs, n_bins, cum=False):
    _set.append(dist)

plot_num = 1
for z in [2, 4, 7]:
    
    t1 = []
    t2 = []
    for d in _set:

        cd = [sum(d[:i+1]) for i in range(len(d))]
        ncd = np.array(cd)/max(cd)
        t1.append(sum(ncd))

        G = [sum(d[:i+1])**(z) for i in range(len(d))]
        G_ = np.array(G)/(n_obs**z)

        t2.append(sum(G_))

    print('n:', n_obs, 'k:', n_bins)
    print('cardinality:', len(t1), '| no. of unique ΣF/n:', 
          len(list(set(t1))), '| no. of unique ΣF^z/n^z:', len(list(set(t2))), '\n')

    ax = plt.subplot(2, 3, plot_num)
    plt.scatter(t1, t2, s=1, c='k')
    plt.xlabel(r'$\sum{F/n}$', fontsize= 10)
    plt.ylabel(r'$\sum{F^{' + str(z) + '}/n^{' + str(z) + '}}$', fontsize= 10)
    
    s = '|A' + r'$_{n' + '=' + str(n_obs) + ', ' + 'k' + '=' + str(n_bins) + '}$' + '| = ' + str(len(_set)) + '\n'
    s += 'Values of ' + r'$\sum{F^{' + str(z) + '}/n^{' + str(z) + '}}$' + '\n = ' + str(len(list(set(t2))))
    
    plt.text(1.01, 3.65, s, fontsize=7)
    plt.tick_params(axis='both', labelsize=7)
    plot_num += 1
    
print('\n')

################  BOTTOM ROW  ###########  

N_obs = [10, 10, 20]
N_bins = [5, 10, 10]
#N_obs = [10, 10, 10]
#N_bins = [5, 5, 5]
sets = []
for i, n_obs in enumerate(N_obs):
    n_bins = N_bins[i]
    
    _set = []
    for dist in generate_cum_dists(n_obs, n_bins, cum=False):
        _set.append(dist)
    sets.append(_set)


for i, n_obs in enumerate(N_obs):
    n_bins = N_bins[i]
    
    _set = sets[i]
    
    t1 = []
    t2 = []
    for d in _set:

        cd = [sum(d[:ii+1]) for ii in range(len(d))]
        ncd = np.array(cd)/max(cd)
        t1.append(sum(ncd))

        z = 1 + 1/n_obs #(n_bins + 1)/n_bins
        G = [sum(d[:ii+1])**(z) for ii in range(len(d))]
        G_ = np.array(G)/(n_obs**z)
        t2.append(sum(G_))

    print('n:', n_obs, 'k:', n_bins)
    print('cardinality:', len(t1), '| no. of unique ΣF/n:', 
          len(list(set(t1))), '| no. of unique ΣF^z/n^z:', len(list(set(t2))), '\n')
    
    s = '|A' + r'$_{n' + '=' + str(n_obs) + ', ' + 'k' + '=' + str(n_bins) + '}$' + '| = ' + str(len(_set)) + '\n'
    s += 'Values of ' + r'$\sum{F^{' + str(z) + '}/n^{' + str(z) + '}}$' + '\n = ' + str(len(_set))
    
    if len(_set) > 10**5: 
        indices = np.random.choice(len(_set), 10**5, replace=False)
        t1 = np.array(t1)
        t2 = np.array(t2)
        t1 = t1[indices]
        t2 = t2[indices]
        t1 = t1.tolist()
        t2 = t2.tolist()
    
    ax = plt.subplot(2, 3, plot_num)
    plt.scatter(t1, t2, s=1, c='k')
    plt.xlabel(r'$\sum{F/n}$', fontsize= 10)
    plt.ylabel(r'$\sum{F^{' + str(z) + '}/n^{' + str(z) + '}}$', fontsize= 10)
    
    if plot_num == 4:
        plt.text(1. * min(t1), 0.73*max(t2), s, fontsize=7)
    elif plot_num == 5:
        plt.text(1. * min(t1), 0.7*max(t2), s, fontsize=7)
    elif plot_num == 6:
        plt.text(0.9 * min(t1), 0.74*max(t2), s, fontsize=7)
        
    plt.tick_params(axis='both', labelsize=7)
    plot_num += 1
    

fig.patch.set_facecolor('white')
plt.subplots_adjust(hspace=0.45, wspace=0.4)
plt.savefig('Final_Figs/manuscript/Fig1.pdf', bbox_inches='tight', format='pdf', dpi=600)
plt.savefig('Final_Figs/manuscript/Fig1.jpg', bbox_inches='tight', format='jpg', dpi=600)
plt.close()

n: 10 k: 5
cardinality: 1001 | no. of unique ΣF/n: 60 | no. of unique ΣF^z/n^z: 355 

n: 10 k: 5
cardinality: 1001 | no. of unique ΣF/n: 60 | no. of unique ΣF^z/n^z: 956 

n: 10 k: 5
cardinality: 1001 | no. of unique ΣF/n: 60 | no. of unique ΣF^z/n^z: 1001 



n: 10 k: 5
cardinality: 1001 | no. of unique ΣF/n: 60 | no. of unique ΣF^z/n^z: 1001 

n: 10 k: 10
cardinality: 92378 | no. of unique ΣF/n: 208 | no. of unique ΣF^z/n^z: 92378 

n: 20 k: 10
cardinality: 10015005 | no. of unique ΣF/n: 491 | no. of unique ΣF^z/n^z: 10015005 

